In [1]:
import spacy
import pandas as pd
from sklearn.model_selection import train_test_split


#### Cargo los Datos

In [2]:
data=pd.read_csv("../data/data.csv")
data.head()

,clasificacion,texto
0,1,El presidente abre la puerta a unos comicios e...
1,1,"En un escrito, la abogada del Estado Rosa Marí..."
2,0,Este lunes y martes la Asamblea de Madrid acog...
3,1,Resulta evidente que la ley ha tenido algunos ...
4,1,"El hispanista, que ya ocupó un puesto simbólic..."


In [3]:
data.value_counts("clasificacion")

clasificacion
1    70908
0    51868
Name: count, dtype: int64

In [4]:
data['texto']= data['texto'].astype(str)

In [5]:
data_chequeado=pd.read_csv("../data/chequeado.csv")
data_chequeado.head()

,clasificacion,texto
0,1,No se divulgaron resultados oficiales previos ...
1,0,Se difundieron en redes sociales supuestos pri...
2,1,"En el balotaje 2023, las boletas con el número..."
3,0,"En el balotaje 2023, las boletas con el número..."
4,0,"Miles de usuarios denuncian que, durante la jo..."


In [10]:
val = data_chequeado.value_counts("clasificacion")
print(val)

clasificacion
0    1770
1     857
Name: count, dtype: int64


In [7]:
data_chequeado['texto']= data_chequeado['texto'].astype(str)

#### Divido en Train, Test Y Split

In [19]:
# Divido el conjunto de datos en entrenamiento (60%), prueba (20%) y validación (20%)
train_data, test_valid_data = train_test_split(data, test_size=0.4, random_state=42)
test_data, val_data = train_test_split(test_valid_data, test_size=0.5, random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(train_data))
print("Tamaño del conjunto de prueba:", len(test_data))
print("Tamaño del conjunto de validación:", len(val_data))

Tamaño del conjunto de entrenamiento: 73665
Tamaño del conjunto de prueba: 24555
Tamaño del conjunto de validación: 24556


In [26]:
train_count = train_data.value_counts("clasificacion")
val_count = val_data.value_counts("clasificacion")
test_count = test_data.value_counts("clasificacion")

print("Valores del conjunto de entrenamiento:", train_count[0], ("(False)"),train_count[1], (" (True) "))
print("Valores del conjunto de prueba:", test_count[0], ("(False)"),test_count[1], (" (True) "))
print("Valores del conjunto de validación:", val_count[0], ("(False)"),val_count[1], (" (True) "))

Valores del conjunto de entrenamiento: 31131 (False) 42534  (True) 
Valores del conjunto de prueba: 10335 (False) 14220  (True) 
Valores del conjunto de validación: 10402 (False) 14154  (True) 


In [27]:
# Divido el conjunto de datos en entrenamiento (60%), prueba (20%) y validación (20%)
train_chequeado, test_valid_chequeado= train_test_split(data_chequeado, test_size=0.4, random_state=42)
test_chequeado, val_chequeado = train_test_split(test_valid_chequeado, test_size=0.5, random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(train_chequeado))
print("Tamaño del conjunto de prueba:", len(test_chequeado))
print("Tamaño del conjunto de validación:", len(val_chequeado))

Tamaño del conjunto de entrenamiento: 1576
Tamaño del conjunto de prueba: 525
Tamaño del conjunto de validación: 526


In [28]:
train_count = train_chequeado.value_counts("clasificacion")
val_count = val_chequeado.value_counts("clasificacion")
test_count = test_chequeado.value_counts("clasificacion")

print("Valores del conjunto de entrenamiento:", train_count[0], ("(False)"),train_count[1], (" (True) "))
print("Valores del conjunto de prueba:", test_count[0], ("(False)"),test_count[1], (" (True) "))
print("Valores del conjunto de validación:", val_count[0], ("(False)"),val_count[1], (" (True) "))

Valores del conjunto de entrenamiento: 1046 (False) 530  (True) 
Valores del conjunto de prueba: 353 (False) 172  (True) 
Valores del conjunto de validación: 371 (False) 155  (True) 


#### Transformo los Datos con Spacy

In [10]:
def convert_pandas_to_tuple(data):
    data = data.to_dict(orient='records')
    return [(d["texto"], {"FALSE": d["clasificacion"] == 0, "TRUE": d["clasificacion"] == 1}) for d in data]

In [11]:
tuple_train_data = convert_pandas_to_tuple(train_data)
tuple_val_data = convert_pandas_to_tuple(val_data)
tuple_test_data = convert_pandas_to_tuple(test_data)

In [12]:
tuple_train_chequeado = convert_pandas_to_tuple(train_chequeado)
tuple_val_chequeado = convert_pandas_to_tuple(val_chequeado)
tuple_test_chequeado = convert_pandas_to_tuple(test_chequeado)

In [ ]:
nlp = spacy.load("es_core_news_lg")

In [14]:
def process_docs(data,nlp):
    docs = []
    for doc, context in nlp.pipe(data, as_tuples=True):
        doc.cats = context
        docs.append(doc)
    return docs

In [ ]:
doc_train_data = process_docs(tuple_train_data,nlp)

In [71]:
doc_test_data = process_docs(tuple_test_data,nlp)

In [72]:
doc_val_data = process_docs(tuple_val_data,nlp)

In [15]:
doc_train_chequeado = process_docs(tuple_train_chequeado,nlp)

In [16]:
doc_test_chequeado = process_docs(tuple_test_chequeado,nlp)

In [17]:
doc_val_chequeado = process_docs(tuple_val_chequeado,nlp)

#### Almaceno los Docs

In [18]:
from spacy.tokens import DocBin

In [21]:
def save_docs(data,path):
    docbin = DocBin(docs=data)
    docbin.to_disk(path)

In [70]:
save_docs(doc_train_data,"./docs/train_data.spacy")

In [73]:
save_docs(doc_test_data,"./docs/test_data.spacy")

In [74]:
save_docs(doc_val_data,"./docs/val_data.spacy")

In [22]:
save_docs(doc_train_chequeado,"./docs/train_chequeado.spacy")

In [23]:
save_docs(doc_test_chequeado,"./docs/test_chequeado.spacy")

In [24]:
save_docs(doc_val_chequeado,"./docs/val_chequeado.spacy")

#### Entreno Modelo

##### Chequeado

In [ ]:
#! python -m spacy train ./config.cfg --output ./models/data_chequeado --paths.train ./docs/train_chequeado.spacy --paths.dev ./docs/val_chequeado.spacy

#============================= Training pipeline =============================
#ℹ Pipeline: ['tok2vec', 'textcat']
#ℹ Initial learn rate: 0.001
#E    #       LOSS TOK2VEC  LOSS TEXTCAT  CATS_SCORE  SCORE 
#---  ------  ------------  ------------  ----------  ------
#  0       0          0.00          0.25       46.25    0.46
#  0     200         16.89         59.00       46.19    0.46
#  1     400         32.86         51.18       40.97    0.41
#  1     600         42.93         52.93       49.28    0.49
#  2     800         27.38         48.80       59.79    0.60
#  3    1000         30.25         48.58       58.42    0.58
#  5    1200         89.07         44.78       63.62    0.64
#  7    1400        236.26         38.71       52.69    0.53
#  9    1600        249.23         40.23       58.92    0.59
# 11    1800        532.75         36.77       58.08    0.58
# 15    2000       1454.30         32.91       60.98    0.61
# 19    2200       4959.55         26.96       55.18    0.55
# 23    2400      10832.40         24.28       63.48    0.63
# 28    2600      23742.59         20.82       57.78    0.58
# 33    2800      37558.84         18.93       56.31    0.56

In [ ]:
#python -m spacy evaluate ./models/model_chequeado/model-best/ ./docs/test_chequeado.spacy

#================================== Results ==================================

#TOK                 100.00
#TEXTCAT (macro F)   64.02 
#SPEED               79    


#=========================== Textcat F (per label) ===========================

#            P       R       F
#FALSE   85.23   57.22   68.47
#TRUE    47.57   79.65   59.57


#======================== Textcat ROC AUC (per label) ========================

#        ROC AUC
#FALSE      0.73
#TRUE       0.73



In [31]:
! python -m spacy benchmark accuracy ./models/model_chequeado/model-best/ ./docs/test_chequeado.spacy --output ./metrics/model_chequeado.json


/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
ℹ Using CPU
/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK                 100.00
TEXTCAT (macro F)   64.02 
SPEED               461   


=========================== Textcat F (per label) ===========================

            P       R       F
FALSE   85.23   57.22   68.47
TRUE    47.57   79.65   59.57


======================== Textcat ROC AUC (per label) ========================

        ROC AUC
FALSE      0.73
TRUE 